In [1]:
import torch
import torchvision
import torch.nn as nn
from torchvision import datasets,transforms,models
from torchvision.models import EfficientNet_V2_S_Weights, SqueezeNet1_1_Weights, ShuffleNet_V2_X0_5_Weights, MobileNet_V3_Small_Weights

In [2]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

## SqueezeNet

In [4]:
sq_model = models.squeezenet1_1(weights=SqueezeNet1_1_Weights.DEFAULT)

In [5]:
print(f"The squeezenet model has {count_parameters(sq_model):,} trainable parameters")

The squeezenet model has 1,235,496 trainable parameters


In [7]:
sq_model

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): MaxPool2d

In [30]:
#freezing previous layers
for param in sq_model.features.parameters():
    param.requires_grad = False

In [31]:
# modifying the last layer to match desired output class
num_classes = 2
in_ftrs = sq_model.classifier[1].in_channels
features = list(sq_model.classifier.children())[:-3] # Remove last 3 layers
features.extend([nn.Conv2d(in_ftrs, num_classes, kernel_size=1)]) # Add
features.extend([nn.ReLU(inplace=True)]) # Add
features.extend([nn.AdaptiveAvgPool2d(output_size=(1,1))]) # Add
sq_model.classifier = nn.Sequential(*features)

In [32]:
last_layer_list_2 = [sq_model.state_dict()['classifier.1.weight'], sq_model.state_dict()['classifier.1.bias']]

In [35]:
weights2 = last_layer_list_2[0]
weights2.size()

torch.Size([2, 512, 1, 1])

## ShuffleNet_v2_x0_5

In [9]:
shf_model = models.shufflenet_v2_x0_5(weights=ShuffleNet_V2_X0_5_Weights.DEFAULT)
print(f"The shuffleenet model has {count_parameters(shf_model):,} trainable parameters")

The shuffleenet model has 1,366,792 trainable parameters


In [13]:
shf_model

ShuffleNetV2(
  (conv1): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (stage2): Sequential(
    (0): InvertedResidual(
      (branch1): Sequential(
        (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=24, bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
      )
      (branch2): Sequential(
        (0): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_

In [20]:
def freeze_all_but_fc(model):
  """
  Freezes the parameters of all layers except the fc layer in a ShuffleNetV2 model.

  Args:
      model: A ShuffleNetV2 model instance.

  Returns:
      The modified ShuffleNetV2 model.
  """
  for name, param in model.named_parameters():
    if name not in ['fc.weight', 'fc.bias']:
        print(name)
        param.requires_grad = False
  return model


In [21]:
model = freeze_all_but_fc(shf_model)

conv1.0.weight
conv1.1.weight
conv1.1.bias
stage2.0.branch1.0.weight
stage2.0.branch1.1.weight
stage2.0.branch1.1.bias
stage2.0.branch1.2.weight
stage2.0.branch1.3.weight
stage2.0.branch1.3.bias
stage2.0.branch2.0.weight
stage2.0.branch2.1.weight
stage2.0.branch2.1.bias
stage2.0.branch2.3.weight
stage2.0.branch2.4.weight
stage2.0.branch2.4.bias
stage2.0.branch2.5.weight
stage2.0.branch2.6.weight
stage2.0.branch2.6.bias
stage2.1.branch2.0.weight
stage2.1.branch2.1.weight
stage2.1.branch2.1.bias
stage2.1.branch2.3.weight
stage2.1.branch2.4.weight
stage2.1.branch2.4.bias
stage2.1.branch2.5.weight
stage2.1.branch2.6.weight
stage2.1.branch2.6.bias
stage2.2.branch2.0.weight
stage2.2.branch2.1.weight
stage2.2.branch2.1.bias
stage2.2.branch2.3.weight
stage2.2.branch2.4.weight
stage2.2.branch2.4.bias
stage2.2.branch2.5.weight
stage2.2.branch2.6.weight
stage2.2.branch2.6.bias
stage2.3.branch2.0.weight
stage2.3.branch2.1.weight
stage2.3.branch2.1.bias
stage2.3.branch2.3.weight
stage2.3.branch2.4.

In [17]:
model

ShuffleNetV2(
  (conv1): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (stage2): Sequential(
    (0): InvertedResidual(
      (branch1): Sequential(
        (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=24, bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
      )
      (branch2): Sequential(
        (0): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_

In [4]:
for param in shf_model.features.parameters():
        param.requires_grad = False

AttributeError: 'ShuffleNetV2' object has no attribute 'features'

In [42]:
model3.fc = nn.Linear(in_features=model3.fc.in_features, out_features=2)

## EfficientNet_v2_s

In [ ]:
eff_model = models.efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.DEFAULT)
print(f"The squeezenet model has {count_parameters(eff_model):,} trainable parameters")

In [16]:
num_features = eff_model.classifier[1].in_features
print(num_features)
eff_model.classifier[1] = nn.Linear(num_features, 2)

1280


In [26]:
last_layer_list_eff = [eff_model.state_dict()['classifier.1.weight'], eff_model.state_dict()['classifier.1.bias']]

In [34]:
weights = last_layer_list_eff[0]
weights.size()

torch.Size([2, 1280])

## SimpleNet

In [10]:
simple = torch.hub.load("coderx7/simplenet_pytorch:v1.0.0", "simplenetv1_small_m2_05", pretrained=True)

/home/anas/anaconda3/envs/pylinenv/lib/python3.10/site-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/coderx7/simplenet_pytorch/zipball/v1.0.0" to /home/anas/.cache/torch/hub/v1.0.0.zip


saving in checkpoint_path:tmp/simplenetv1_small_m2_05-ca4b3e2b.pth


100%|███████████████████████████████████████| 5.78M/5.78M [00:13<00:00, 444kB/s]


In [12]:
print(f"The simple model has {count_parameters(simple):,} trainable parameters")

The simple model has 1,506,792 trainable parameters


In [11]:
simple

SimpleNet(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.05, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Dropout2d(p=0.0, inplace=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.05, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): Dropout2d(p=0.0, inplace=False)
    (8): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.05, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Dropout2d(p=0.0, inplace=False)
    (12): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (13): BatchNorm2d(64, eps=1e-05, momentum=0.05, affine=True, track_running_stats=True)
    (14): ReLU(inplace=True)
    (15): Dropout2d(p=0.0, inplace=False)
    (16): Conv2d(64, 6

## MobileNet_v3_small

In [3]:
mobile_model = models.mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.DEFAULT)
print(f"The mobilenet_v3_small model has {count_parameters(mobile_model):,} trainable parameters")

Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /home/anas/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth
100%|██████████████████████████████████████| 9.83M/9.83M [00:02<00:00, 4.26MB/s]


The mobilenet_v3_small model has 2,542,856 trainable parameters


In [4]:
mobile_model

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

In [5]:
for param in mobile_model.features.parameters():
    param.requires_grad = False

In [ ]:
# Newly created modules have require_grad=True by default
num_features = pretrained_model_ImageNet.classifier[1].in_channels
features = list(pretrained_model_ImageNet.classifier.children())[:-3] # Remove last 3 layers
features.extend([nn.Conv2d(num_features, n_classes, kernel_size=1)]) # Add
features.extend([nn.ReLU(inplace=True)]) # Add
features.extend([nn.AdaptiveAvgPool2d(output_size=(1,1))]) # Add
pretrained_model_ImageNet.classifier = nn.Sequential(*features)

In [9]:
num_features = mobile_model.classifier[3].in_features
mobile_model.classifier[3] = nn.Linear(num_features, 2)
mobile_model

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

## SqueezeNext

In [3]:
from pytorchcv.model_provider import get_model as ptcv_get_model
net = ptcv_get_model('sqnxt23_w1', pretrained=True)

In [4]:
print(f"The squeezenext model has {count_parameters(net):,} trainable parameters")

The squeezenext model has 724,056 trainable parameters


In [5]:
num_features = net.output.in_features
net.output = nn.Linear(num_features, 2)